In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import os

## Spark to access gcs

we need cloud storage connector [🔗](https://cloud.google.com/dataproc/docs/concepts/connectors/cloud-storage)

we should download the `shaded` version then specify with
```python
.set('spark.jars', jars)
```

there's another possible way
```
.set('spark.jars.packages', 'com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.7')
```
spark will search the maven repository for the package, it sounds great but I encountered problem like `MethodNotFound`, because I've other packages already, and the it kinded of mess up with the dependency...

maybe that's way the official document suggest us to use `shaded` version.

In [2]:
credentials_location = os.path.expanduser('~/.credentials/google/dtalk_de_zcamp/premium-aloe-357506-8c578ad240d3.json')
jars = ','.join(os.path.expanduser(f'~/.ivy2/jars/{x}') for x in [
    'gcs-connector-hadoop3-2.2.7-shaded.jar'
])
conf = (
    SparkConf()
    .setMaster('local[*]')
    .setAppName('test')
    # .set('spark.jars.packages', 'com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.7')
    .set('spark.jars', jars)
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)
    .set("spark.hadoop.fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    .set("spark.hadoop.fs.gs.auth.service.account.json.keyfile", credentials_location)
    .set("spark.hadoop.fs.gs.auth.service.account.enable", "true")

)
spark = (
    SparkSession.builder
        .config(conf=conf)
).getOrCreate()

22/08/06 21:56:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_zones = spark.read.csv('gs://dtc_data_lake_premium-aloe-357506/raw/taxi+_zone_lookup.csv', header='true')
df_zones.show(5)

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



# Similar notes
- [spark on s3](../s3/spark.ipynb)